In [1]:
import torch
from torch.utils.data import DataLoader

import pandas as pd
import random
import numpy as np
import dataset
from tqdm import tqdm
from PIL import Image

import torchvision
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [2]:
random_seed = 11

torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True # 고정하면 학습이 느려진다고 합니다.
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.7.1].
device:[cuda:0].


In [4]:
df = pd.read_csv("../input/data/eval/info_new.csv")
transform = dataset.BaseAugmentation()
test_data = dataset.CustomDataset(df,"label",transform)
test_loader = DataLoader(test_data,batch_size=128,shuffle=False)

In [5]:
test_name = 'T2(resnet50)'
target = ['mask','sex','age']
SD = ['005_0.9989.pt','006_0.9997.pt','005_0.9963.pt']
data = {'mask':[],'sex':[],'age':[]}
for t,s in zip(target,SD):
    model_ = torch.load(f'./model/{test_name}/{t}.pt')
    model_.load_state_dict(torch.load(f'./model/{test_name}/{t}_state_dict/' + s))
    model_.to(device)
    for td in tqdm(test_loader):
        pred = model_(td[0].to(device))
        pred = pred.argmax(dim=-1)
        data[t].extend(pred.cpu().numpy())
        

  0%|          | 0/99 [00:01<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 384.00 MiB (GPU 0; 31.75 GiB total capacity; 30.37 GiB already allocated; 223.50 MiB free; 30.40 GiB reserved in total by PyTorch)

In [ ]:
class_df = pd.DataFrame(data)
class_df.head(10)

,mask,sex,age
0,2,0,1
1,0,0,1
2,2,0,1
3,2,0,1
4,2,0,0
5,0,0,0
6,1,0,1
7,0,1,1
8,0,1,1
9,0,1,0


In [ ]:
class_df['answer'] = class_df['mask']*6 + class_df['sex']*3 + class_df['age']
class_df.head()

,mask,sex,age,answer
0,2,0,1,13
1,0,0,1,1
2,2,0,1,13
3,2,0,1,13
4,2,0,0,12


In [ ]:
df3 = pd.read_csv("../input/data/eval/info.csv")
df3.head()

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0


In [ ]:
df3['ans'] = class_df['answer']
df3.head()

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12


In [ ]:
import os
df3.to_csv(os.path.join('submission/',f'{test_name}_submission.csv'), index=False)